In [ ]:
import requests
import openai
from environment import APEnv
from wrapper import LoggingWrapper

In [ ]:
def llm(prompt, stop=["\n"]):
    response = openai.Completion.create(
        model="text-davinci-002",  # Update to a more recent model if available
        prompt=prompt,
        temperature=0,
        max_tokens=100,
        top_p=1,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=stop
    )
    return response["choices"][0]["text"]


In [ ]:

def get_alert_info(idx):
    # Replace with actual API call to Alert Manager
    # Example response structure:
    # {"device_id": "AP-123", "description": "AP-123 is unresponsive."}
    try:
        response = requests.get(f"https://alert_manager/api/alerts/{idx}")
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Failed to retrieve alert info for index {idx}: {str(e)}")
        return {"device_id": "AP-123", "description": "AP-123 is unresponsive."}


In [ ]:

def step_env(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1
            print(f"Timeout encountered. Retrying action: {action} (Attempt {attempts})")
    return "Action timed out after 10 attempts.", 0, True, {}

def ap_diagnostic(idx=None, prompt=initial_prompt, to_print=True):
    # Fetch alert info based on idx
    alert_info = get_alert_info(idx)
    observation, info = env.reset(alert_info=alert_info, return_info=True)
    if to_print:
        print(f"Alert: {observation}")
    
    prompt += observation + "\n"
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):  # Limit to 7 steps
        n_calls += 1
        thought_action = llm(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except ValueError:
            print('Parsing error:', thought_action)
            n_badcalls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
        
        # Ensure action starts with lowercase
        if not action:
            self.obs = "No action provided."
            continue
        action = action[0].lower() + action[1:]
        
        # Execute Action
        obs, reward, done, info = step_env(env, action)
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if to_print:
            print(step_str)
        
        if done:
            break
    
    if not done:
        # If not done after max steps, force finish
        obs, reward, done, info = step_env(env, "Finish[Diagnostic session completed.]")
        step_str = f"Thought {i+1}: Completing diagnostic.\nAction {i+1}: Finish[Diagnostic session completed.]\nObservation {i+1}: {obs}\n"
        prompt += step_str
        if to_print:
            print(step_str)
    
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return info

# Initialize environment with wrappers
env = APEnv()
env = LoggingWrapper(env, folder="ap_diagnostics_logs")

# Define initial prompt with action definitions and examples
initial_prompt = """You are a network alert diagnostic assistant. Your task is to diagnose Access Point (AP) related alerts by reasoning through the problem and performing actions as needed. Use the following actions:

1. GetDeviceInfo[device_id] - Retrieve information about the device.
2. LookupTopology[device_id] - Check the network topology for the device.
3. RebootDevice[device_id] - Reboot the device.
4. SearchThreatIntel[ip_address] - Search threat intelligence for the IP address.
5. ChangeAPConfiguration[device_id, param1=value1, param2=value2] - Change AP configuration settings.
6. Finish[summary] - Provide a final summary and conclude the diagnostic.

Here is an example diagnostic session:

Thought 1: The AP is unresponsive. I will retrieve its device information.
Action 1: GetDeviceInfo[AP-123]
Observation 1: Device AP-123: Status - offline, Firmware - v2.2.0, Last Seen - 10 minutes ago.

Thought 2: The device is offline. I will check the network topology.
Action 2: LookupTopology[AP-123]
Observation 2: Device AP-123 is connected to Controller CTRL-10, Switch SW-101, Neighbors AP-122, AP-124.

Thought 3: Other APs on the same switch are functioning. I will attempt to reboot AP-123.
Action 3: RebootDevice[AP-123]
Observation 3: Reboot command sent to Device AP-123.

Thought 4: Awaiting AP-123 to come back online.
Action 4: GetDeviceInfo[AP-123]
Observation 4: Device AP-123: Status - online, Firmware - v2.2.0, Last Seen - now.

Thought 5: AP-123 is back online. Diagnostic completed successfully.
Action 5: Finish[AP-123 was successfully rebooted and is now online.]
Observation 5: Episode finished, reward = 1
"""

# Example usage: Diagnosing a specific alert
alert_indices = [101, 202, 303, 404, 505]  # Replace with actual alert IDs

for idx in alert_indices:
    print(f"Starting diagnostic for Alert {idx}")
    info = ap_diagnostic(idx=idx, prompt=initial_prompt, to_print=True)
    print(f"Diagnostic Result for Alert {idx}: {info['answer']}")
    print('-----------\n')
